<img src='../img/bimbo.png'>

- Task: Demand = Sales - Returns
- Create holdout test set

## Data exploration

In [64]:
import pandas as pd
import numpy as np
import os
dir_sub = "C:/Users/568377/Cognizant/Data Science/kaggle/Bimbo Bread/submissions/"
dir_raw = "C:/Users/568377/Cognizant/Data Science/kaggle/Bimbo Bread/raw/"

In [228]:
subm = pd.read_csv(os.path.join(dir_raw,"sample_submission.csv"))
prod = pd.read_csv(os.path.join(dir_raw,"producto_tabla.csv"))
town_state = pd.read_csv(os.path.join(dir_raw,"town_state.csv"))
client = pd.read_csv(os.path.join(dir_raw,"cliente_tabla.csv"))

In [66]:
test = pd.read_csv(os.path.join(dir_raw,"test.csv"), nrows=500000)
train = pd.read_csv(os.path.join(dir_raw,"train.csv"), nrows=500000)

### Features

#### Product

In [229]:
prod.head() #TO BE DONE

,Producto_ID,NombreProducto
0,0,NO IDENTIFICADO 0
1,9,Capuccino Moka 750g NES 9
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41
3,53,Burritos Sincro 170g CU LON 53
4,72,Div Tira Mini Doradita 4p 45g TR 72


#### Town/State

In [230]:
town_state.head()

,Agencia_ID,Town,State
0,1110,2008 AG. LAGO FILT,"MÉXICO, D.F."
1,1111,2002 AG. AZCAPOTZALCO,"MÉXICO, D.F."
2,1112,2004 AG. CUAUTITLAN,ESTADO DE MÉXICO
3,1113,2008 AG. LAGO FILT,"MÉXICO, D.F."
4,1114,2029 AG.IZTAPALAPA 2,"MÉXICO, D.F."


In [231]:
#lowercase
town_state['Town'] = town_state['Town'].str.lower()
town_state['State'] = town_state['State'].str.lower()
#separate zip, town_name
town_state['zip'] = town_state['Town'].str[:4]
town_state['town_name'] = town_state['Town'].str[5:]

In [232]:
town_state.head()

,Agencia_ID,Town,State,zip,town_name
0,1110,2008 ag. lago filt,"méxico, d.f.",2008,ag. lago filt
1,1111,2002 ag. azcapotzalco,"méxico, d.f.",2002,ag. azcapotzalco
2,1112,2004 ag. cuautitlan,estado de méxico,2004,ag. cuautitlan
3,1113,2008 ag. lago filt,"méxico, d.f.",2008,ag. lago filt
4,1114,2029 ag.iztapalapa 2,"méxico, d.f.",2029,ag.iztapalapa 2


In [233]:
len(town_state['Town'].unique()), len(town_state['zip'].unique()), len(town_state['town_name'].unique())
##town_state = town_state.groupby('zip')
##town_state['Town'].str[4].unique() -> clean

(259, 257, 257)

In [234]:
# 2 times similar zips have different town_names
df = town_state.groupby('zip').Town.nunique()
df[df==2] #3 entries with different town names
double_town_names = np.array(df[df==2].index)
double_town_names # zips

array(['2152', '2169'], dtype=object)

In [235]:
town_state['zip'].max() #max zip number

'3216'

In [236]:
town_state[town_state['zip'] == double_town_names[0]]

,Agencia_ID,Town,State,zip,town_name
199,1524,2152 cruce de anden noroeste,baja california norte,2152,cruce de anden noroeste
371,2022,2152 altamirano_bm,guerrero,2152,altamirano_bm
583,21529,2152 altamirano_bm,guerrero,2152,altamirano_bm


In [237]:
town_state.loc[199,'zip']=9999
town_state.loc[199,'Town']="9999 cruce de anden noroeste"
town_state[town_state['zip'] == 9999]

,Agencia_ID,Town,State,zip,town_name
199,1524,9999 cruce de anden noroeste,baja california norte,9999,cruce de anden noroeste


In [238]:
town_state[town_state['zip'] == double_town_names[1]]

,Agencia_ID,Town,State,zip,town_name
311,1694,2169 cruce de andén sureste,puebla,2169,cruce de andén sureste
372,2023,2169 pinotepa,oaxaca,2169,pinotepa
599,21699,2169 pinotepa,oaxaca,2169,pinotepa


In [239]:
town_state.loc[311,'zip']=9998
town_state.loc[311,'Town']="9998 cruce de andén sureste"
town_state[town_state['zip'] == 9998]

,Agencia_ID,Town,State,zip,town_name
311,1694,9998 cruce de andén sureste,puebla,9998,cruce de andén sureste


In [240]:
town_state.head()

,Agencia_ID,Town,State,zip,town_name
0,1110,2008 ag. lago filt,"méxico, d.f.",2008,ag. lago filt
1,1111,2002 ag. azcapotzalco,"méxico, d.f.",2002,ag. azcapotzalco
2,1112,2004 ag. cuautitlan,estado de méxico,2004,ag. cuautitlan
3,1113,2008 ag. lago filt,"méxico, d.f.",2008,ag. lago filt
4,1114,2029 ag.iztapalapa 2,"méxico, d.f.",2029,ag.iztapalapa 2


In [241]:
len(town_state['Town'].unique()), len(town_state['zip'].unique()), len(town_state['town_name'].unique())

(259, 259, 257)

In [242]:
df1 = town_state.groupby('town_name').zip.nunique()

In [243]:
df1[df1==2]

town_name
los mochis    2
pinotepa      2
Name: zip, dtype: int64

In [244]:
df1[df1==2] #3 entries with different town names
double_zip_names = np.array(df1[df1==2].index)
double_zip_names # zips

array(['los mochis', 'pinotepa'], dtype=object)

In [245]:
town_state[town_state['town_name'] == double_zip_names[0]] #towns can have multiple zip codes!!! :)

,Agencia_ID,Town,State,zip,town_name
211,1544,2561 los mochis,sinaloa,2561,los mochis
227,1564,2561 los mochis,sinaloa,2561,los mochis
248,1598,2561 los mochis,sinaloa,2561,los mochis
498,2655,2655 los mochis,sinaloa,2655,los mochis
774,25619,2561 los mochis,sinaloa,2561,los mochis


In [246]:
town_state[town_state['town_name'] == double_zip_names[1]] #ID 387 interferes with town_name. If town_name big effect, look here. Test here for improvement.

,Agencia_ID,Town,State,zip,town_name
372,2023,2169 pinotepa,oaxaca,2169,pinotepa
387,2055,2055 pinotepa,puebla,2055,pinotepa
599,21699,2169 pinotepa,oaxaca,2169,pinotepa


In [247]:
town_state.head()

,Agencia_ID,Town,State,zip,town_name
0,1110,2008 ag. lago filt,"méxico, d.f.",2008,ag. lago filt
1,1111,2002 ag. azcapotzalco,"méxico, d.f.",2002,ag. azcapotzalco
2,1112,2004 ag. cuautitlan,estado de méxico,2004,ag. cuautitlan
3,1113,2008 ag. lago filt,"méxico, d.f.",2008,ag. lago filt
4,1114,2029 ag.iztapalapa 2,"méxico, d.f.",2029,ag.iztapalapa 2


# NEXT STEP

In [ ]:
# cd. ag. separate them and create new column with classifier

In [252]:
sorted(town_state['town_name'].str[:4].unique())

['acap',
 'acay',
 'acuñ',
 'aero',
 'ag. ',
 'ag.h',
 'ag.i',
 'ag.m',
 'ag.p',
 'ag.t',
 'agen',
 'agua',
 'alta',
 'apan',
 'apat',
 'apiz',
 'arri',
 'atla',
 'atot',
 'autl',
 'azca',
 'bi_s',
 'blm_',
 'bw_g',
 'cabo',
 'camp',
 'cana',
 'canc',
 'card',
 'casa',
 'cava',
 'cd g',
 'cd. ',
 'cela',
 'chal',
 'chet',
 'chih',
 'chil',
 'ciud',
 'coat',
 'coli',
 'coma',
 'comi',
 'conv',
 'cord',
 'cozu',
 'cruc',
 'cuah',
 'cuau',
 'cubi',
 'cuer',
 'culi',
 'deli',
 'dura',
 'emil',
 'ense',
 'esca',
 'esco',
 'fres',
 'gome',
 'gonz',
 'gran',
 'guad',
 'guam',
 'guay',
 'guer',
 'herm',
 'hida',
 'huaj',
 'igua',
 'indu',
 'irap',
 'isla',
 'ixte',
 'izuc',
 'jala',
 'joju',
 'juar',
 'juxt',
 'la f',
 'la p',
 'lago',
 'lare',
 'las ',
 'laza',
 'leon',
 'libe',
 'lina',
 'linc',
 'los ',
 'manz',
 'mara',
 'mari',
 'mart',
 'mata',
 'mate',
 'maza',
 'meri',
 'mexi',
 'mina',
 'mitr',
 'moct',
 'monc',
 'more',
 'nauc',
 'navo',
 'noga',
 'nort',
 'nuev',
 'oaxa',
 'ocot',
 

In [255]:
sorted(town_state['town_name'].str[:12].unique())

['acaponeta',
 'acapulco cos',
 'acapulco esc',
 'acapulco san',
 'acayucan',
 'acuña',
 'aeropuerto',
 'ag. atizapan',
 'ag. azcapotz',
 'ag. centro',
 'ag. ceylan',
 'ag. coacalco',
 'ag. cuautitl',
 'ag. ixtapalu',
 'ag. iztapala',
 'ag. la villa',
 'ag. lago fil',
 'ag. los reye',
 'ag. mega nau',
 'ag. mixcoac',
 'ag. neza',
 'ag. pantitla',
 'ag. rojo gom',
 'ag. san anto',
 'ag. san lore',
 'ag. santa cl',
 'ag. santa lu',
 'ag. tepalcat',
 'ag. tepeji d',
 'ag. tepozotl',
 'ag. texcoco',
 'ag. tizayuca',
 'ag. tultitla',
 'ag. vallejo',
 'ag. xalostoc',
 'ag. xochimil',
 'ag.huejutla',
 'ag.iztapalap',
 'ag.mixquiahu',
 'ag.pachuca',
 'ag.tulancing',
 'agencia suan',
 'agua prieta',
 'aguascalient',
 'altamirano_b',
 'apan',
 'apatzingan',
 'apizaco mari',
 'arriaga_bm',
 'atlacomulco',
 'atotonilco',
 'autlan_bm',
 'azcapotzalco',
 'bi_san franc',
 'blm_ag. camp',
 'bw_guasave',
 'caborca',
 'campeche',
 'cananea',
 'cancun ii',
 'cancun_bm',
 'cardenas',
 'casas grande',
 'ca

#### Client

In [257]:
client.head()

,Cliente_ID,NombreCliente
0,0,SIN NOMBRE
1,1,OXXO XINANTECATL
2,2,SIN NOMBRE
3,3,EL MORENO
4,4,SDN SER DE ALIM CUERPO SA CIA DE INT


In [153]:
test.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238
2,2,10,2045,1,2831,4549769,32940
3,3,11,1227,1,4448,4717855,43066
4,4,11,1219,1,1130,966351,1277


In [154]:
train.head()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.14,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.52,0,0.0,4
2,3,1110,7,3301,15766,1238,4,39.32,0,0.0,4
3,3,1110,7,3301,15766,1240,4,33.52,0,0.0,4
4,3,1110,7,3301,15766,1242,3,22.92,0,0.0,3


# Feature engineering

https://www.kaggle.com/cast42/rossmann-store-sales/xgboost-in-python-with-rmspe-v2/code

In [2]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import operator
import matplotlib

ImportError: No module named 'xgboost'

# sub 2

In [85]:
train = pd.read_csv(os.path.join(dir_raw,"sample_submission.csv"))

In [86]:
train["Demanda_uni_equil"]=6

In [87]:
train.to_csv(os.path.join(dir_sub, "submission_2.csv"), index=False)